# Dog vs Cat

## Extracción de características 2

Importamos algunas librerías que vamos a utilizar

In [2]:
import numpy as np
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 

Instalamos tensorflow si no lo tenemos instalado anteriormente

In [ ]:
!pip install tensorflow

Inicializamos variables

En este caso utilizaremos el dataset `cat_dog_100` pero en cualquier momento se puede cambiar

In [19]:
rutaDogsTrain = 'cat_dog_100/train/dog'
rutaDogsTest = 'cat_dog_100/test/dog'
rutaCatsTrain = 'cat_dog_100/train/cat'
rutaCatsTest = 'cat_dog_100/test/cat'

tamanoMax = 256
sizeVentana = 16
tamanoMax = int(tamanoMax/sizeVentana)*sizeVentana

Se recorren los directorios para obtener los nombres de las imágenes

In [20]:
from os import walk

dogTrain = []
dogTest = []
catTest = []
catTrain = []
for (dirpath, dirnames, filenames) in walk(rutaDogsTrain):
    dogTrain.extend(filenames)
    
for (dirpath, dirnames, filenames) in walk(rutaDogsTest):
    dogTest.extend(filenames)

for (dirpath, dirnames, filenames) in walk(rutaCatsTest):
    catTest.extend(filenames)

for (dirpath, dirnames, filenames) in walk(rutaCatsTrain):
    catTrain.extend(filenames)
    
print(f'{rutaDogsTrain}/{dogTrain[0]}')

cat_dog_100/train/dog/dog.10000.jpg


La función `resize_image` redimensiona la imagen en *tamanoMax x tamanoMax*.

Probaremos a recortar la imagen para no deformarlas y así comprobar si obtenemos un mejor resultado.

In [21]:
def recorta_imagen(imagen, n):
    """
    Esta función recibe una imagen y un tamaño n y recorta la imagen de tal manera que el número final de filas/columnas 
    sea múltiplo de n. La funció debe recortar las filas de la derecha/abajo.
    """
    size = np.min([int(imagen.shape[0]/n)*n, int(imagen.shape[1]/n)*n])
    return imagen[:size, :size]

def resize_image(imagen, sizeVentana, tamanoMax, recorta=False):
    if(recorta): imagen = recorta_imagen(imagen, sizeVentana)
    imagen2 = cv2.resize(imagen, [tamanoMax,tamanoMax])
    return imagen2

Función para aplicar un filtro

In [22]:
def filtro(imagen, mascara):
    a = mascara.shape[0]//2
    b = mascara.shape[1]//2
    imagen_ampliada = cv2.copyMakeBorder(imagen, a, a, b, b, cv2.BORDER_REPLICATE)
    imagen2 = np.zeros(imagen.shape)
    for i in range(a, imagen.shape[0] + a):
        for j in range(b, imagen.shape[1] + b):
            imagen2[i-a, j-b] = np.sum(imagen_ampliada[i-a:i+a+1, j-b:j+b+1] * mascara)
            
    return imagen2

Función para aplicar un filtro espacial de aproximación de la derivada parcial y obtenemos Phi y E

In [25]:
def gradiente(imagen):
    m_g_x = np.array([[0, -1, 0], [0, 0, 0], [0, 1, 0]])
    m_g_y = np.array([[0, 0, 0], [-1, 0, 1], [0, 0, 0]])
    Ix = filtro(imagen, m_g_x)
    Iy = filtro(imagen, m_g_y)
    E = np.sqrt(Ix**2 + Iy**2)
    Phi = np.rad2deg(np.arctan2(Iy, Ix))
    
    return E, Phi

La función `transforma_imagen` divide en celdas de tamaños sizeVentana y contruimos un histograma para cada celda en función de Phi y E 

In [26]:
def transforma_bloque(celdaPhi, celdaE, bins = 9):
    celdaPhi = np.abs(celdaPhi)
    hp = cv2.calcHist([np.uint8(celdaPhi)], [0], None, [bins], [0, 180])
    a = 180/bins 
    for i in range(bins):
        hp[i] += np.sum(celdaE[(celdaPhi > a*i) & (celdaPhi < -a*(i+1))])
    hp = hp / np.sqrt(np.sum(hp**2)) #Normalizar
    return np.ravel(hp)

In [27]:
def transforma_imagen(imagen, sizeVentana, tamanoMax):
    E, Phi = gradiente(imagen)
    Phi = resize_image(Phi, sizeVentana, tamanoMax, False)  #Mejor recortar a True
    E = resize_image(E, sizeVentana, tamanoMax, False) #Mejor recortar a True
    caracteristicas = []
    a = int(Phi.shape[0]/sizeVentana)
    for i in range(a):
        for j in range(a):
            celdaPhi = Phi[i*sizeVentana:(i+1)*sizeVentana,j*sizeVentana:(j+1)*sizeVentana]
            celdaE = E[i*sizeVentana:(i+1)*sizeVentana,j*sizeVentana:(j+1)*sizeVentana]
            h = transforma_bloque(celdaPhi, celdaE)  
            caracteristicas.append(h)
    return np.concatenate(caracteristicas)

Extraemos las características de nuestro conjunto de imágenes (Train y Test)

In [29]:
X = []
m = 2
for i in tqdm(range(len(dogTrain)), desc = "Perros", colour = '#FF88F8'):
    imagen = cv2.imread(f'{rutaDogsTrain}/{dogTrain[i]}',0)
    caracteristicas = transforma_imagen(imagen, sizeVentana, tamanoMax)
    X.append(caracteristicas)

for i in tqdm(range(len(catTrain)), desc = "Gatos", colour = '#88FFDD'):
    imagen = cv2.imread(f'{rutaCatsTrain}/{catTrain[i]}',0)
    caracteristicas = transforma_imagen(imagen, sizeVentana, tamanoMax)
    X.append(caracteristicas)
    
X = np.array(X)
y = np.hstack((np.ones(len(dogTrain)), np.zeros(len(catTrain)))).reshape(-1, 1)


Gatos: 100%|█████████████████████████████████████████████████████████████████████████| 100/100 [05:34<00:00,  3.34s/it]


In [ ]:
Xtest = []
for i in tqdm(range(len(dogTest)), desc = "Perros Test", colour = '#FF88BB'):
    imagen = cv2.imread(f'{rutaDogsTest}/{dogTest[i]}',0)
    caracteristicas = transforma_imagen(imagen, sizeVentana, tamanoMax)
    Xtest.append(caracteristicas)

for i in tqdm(range(len(catTest)), desc = "Gatos Test", colour = '#88FFE0'):
    imagen = cv2.imread(f'{rutaCatsTest}/{catTest[i]}',0)
    caracteristicas = transforma_imagen(imagen, sizeVentana, tamanoMax)
    Xtest.append(caracteristicas)
    
Xtest = np.array(Xtest)
ytest = np.hstack((np.ones(len(dogTest)), np.zeros(len(catTest)))).reshape(-1, 1)

Perros Test:  10%|███████                                                               | 2/20 [00:02<00:21,  1.17s/it]

Particionamos el conjunto de datos para obtener el conjunto de validación (15%)

In [ ]:
Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size = 0.15, random_state = 42)
print(Xtrain.shape)

Importamos las liberías para usar redes neuronales

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, LSTM, TimeDistributed, GRU, SimpleRNN
import tensorflow.keras as keras

print(tf.config.list_physical_devices())
print("Número de GPUs disponibles: ", len(tf.config.experimental.list_physical_devices('GPU')))

Creamos un modelo simple porque no nos vamos a centrar en redes neuronales, más adelante utilizaremos SVMs.

In [ ]:
model = Sequential([
    Dense(32, input_shape = (16384,)),
    Dropout(0.28),
    Dense(32),
    Dense(1, activation = 'sigmoid')
])
model.summary()
model.compile(optimizer = keras.optimizers.Adamax(), loss = keras.losses.binary_crossentropy, metrics = ['acc'])

In [ ]:
historico = model.fit(
    Xtrain, 
    ytrain, 
    batch_size = 1000, 
    epochs = 64,
    validation_data= (Xval, yval)
)

In [ ]:
historial = historico.history
plt.title('Gráfica de coste')
plt.xlabel('Epochs')
plt.ylabel('Coste')
plt.plot(historial['loss'])
plt.plot(historial['val_loss'])
plt.legend(["Train", "Val"]);

In [ ]:
plt.title('Gráfica de accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.plot(historial['acc'])
plt.plot(historial['val_acc'])
plt.legend(["Train", "Val"]);

In [ ]:
prediccion = model.predict(Xtest)
aciertos = np.sum(np.round(prediccion) == ytest)
print(f'Precisión en test: {aciertos/ytest.shape[0]*100}%')

## SVMs

In [ ]:
# Importamos las SVMs
from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Creamos un clasificador SVM usando svm.SVC con kernel='linear', C = 1.0
clf = make_pipeline(StandardScaler(),  svm.SVC(kernel = 'linear', C = 1))
#clf = make_pipeline(StandardScaler(),  svm.SVC(C = 2))

# Entrenamos el modelo
clf.fit(X, y.ravel())
# Obtenemos la precisión
accTrain = clf.score(X, y)
print("Precisión en train: {}%".format(accTrain*100))

print(f"Precisión en test: {clf.score(Xtest, ytest)*100}%")

#### Hemos obtenido un 90% de precisión en Test con ambos modelos.